## Introduction

Réalisez une application de recommandation de contenu projetc OC_IA_P10_Recommandation_contenu. Le projet consiste à créer une application qui recommande 5 articles à un utilisateur en fonction de ses interactions passées avec d'autres articles.

My Content, une jeune start-up, se lance dans la création d'une application de recommandation de contenus pour encourager la lecture. En tant que CTO et co-fondateur de cette entreprise, nous sommes en plein développement de notre premier MVP. Pour cela, nous utiliserons des données en ligne, car nous n'avons pas encore de données utilisateurs.  
Notre objectif principal est de fournir à chaque utilisateur une sélection de cinq articles personnalisés.  
Notre mission comprend :
* le développement d'un système de recommandation avec Azure Functions,
* la création d'une interface utilisateur conviviale,
* le stockage des scripts sur GitHub,
* la réflexion sur l'architecture technique et le système de recommandation.  

Nous devrons également envisager une architecture évolutive pour intégrer de nouveaux utilisateurs et articles.

### Organisation du projet et architecture technique

### Trois dépôts GitHub  :

1. **OC_IA_P10_Recommandation_contenu**  
   - **Lien GitHub** : [https://github.com/preudh/OC_IA_P10_Recommandation_contenu](https://github.com/preudh/OC_IA_P10_Recommandation_contenu)  
   - **Rôle** : c’est ici que je développe et teste mon Notebook (Jupyter). J’y explore différentes méthodes de recommandation, j’y entraîne mes modèles (notamment ALS), et je documente mes résultats. Il fait office de “laboratoire” de recherche et développement.

2. **OC_IA_P10_RecoFunction**  
   - **Lien GitHub** : [https://github.com/preudh/OC_IA_P10_RecoFunction](https://github.com/preudh/OC_IA_P10_RecoFunction)  
   - **Rôle** : ce dépôt contient mon projet **Azure Functions**, qui constitue le “back-end serverless”. J’y charge le modèle entraîné ainsi que les fichiers `.npz` (matrices de popularité, paramètres ALS, etc.), je récupère le CSV des interactions (via Azure Blob Storage) et j’expose une _API_ de recommandation. Cette API est déclenchée à la demande lorsqu’un utilisateur (ou l’appli Streamlit) envoie une requête.

3. **OC_IA_P10_STREAMLIT_APP**  
   - **Lien de l’app déployée** : [https://p10-streamlit-app-2025.azurewebsites.net/](https://p10-streamlit-app-2025.azurewebsites.net/)  
   - **Rôle** : c’est l’interface _Streamlit_, qui sert de “front-end” à mon système. Grâce à ce dépôt, je propose une interface conviviale pour sélectionner un `user_id`, puis effectuer un appel HTTP à la fonction Azure (hébergée dans **OC_IA_P10_RecoFunction**) afin de récupérer et d’afficher les articles recommandés.
   - **Lien GitHub** : https://github.com/preudh/OC_IA_P10_streamlit_app

---

### Comment s’articule l’architecture serverless ?

- **Azure Functions** me permet de déployer une API sans gérer de serveur complet. Je ne suis facturé qu’à l’usage, et Azure s’occupe automatiquement de la montée en charge (scaling). Dans le code de ce dépôt, j’ai une fonction (`__init__.py`) qui se charge de charger le modèle (ALS) et de retourner les recommandations.  
- **Azure Blob Storage** stocke mes fichiers essentiels (CSV d’interactions, matrices, etc.). Ainsi, chaque composant (la fonction Azure, l’app Streamlit) peut récupérer les données en temps réel à partir du même conteneur.  
- **Application Streamlit** : l’utilisateur final n’a accès qu’à l’interface Streamlit. Quand il clique sur “Obtenir des recommandations”, l’app envoie un appel REST à l’API hébergée par Azure Functions. L’API exécute le moteur de recommandation, puis renvoie la liste des articles.

En résumé :

- **OC_IA_P10_Recommandation_contenu** : mon Notebook de test et de recherche pour trouver et entraîner le meilleur modèle.  
- **OC_IA_P10_RecoFunction** : la fonction Azure qui sert d’API, interagit avec le Blob Storage et renvoie les recommandations.  
- **OC_IA_P10_STREAMLIT_APP** : l’interface utilisateur développée avec Streamlit pour faciliter l’accès à mon moteur de recommandation.

Cette séparation clarifie la logique :  
- un dépôt pour la R &D (Notebook),  
- un pour la logique de calcul et d’API (Azure Functions),  
- un pour l’interface (Streamlit).


## Table des matières

1. [Présentation générale du jeu de données](#presentation)  
    1.1 [Chargement des données](#presentation-data)  
    1.2 [Inspection des données](#presentation-inspection)   
2. [Analyse des données](#analyse)  
    2.1 [Valeurs aberrantes](#analyse-outliers)  
    2.2 [Analyse univariée](#analyse-uni)   
    2.3 [Analyse multivariée](#analyse-multi)
3. [Préparation du jeu de données d'étude](#preparation)  
    3.1 [Fusion des données](#preparation-fusion)  
    3.2 [Élaboration d'un indicateur de popularité pour des articles](#preparation-score)   
    3.3 [Séparation du jeu de donnée](#preparation-split)   
4. [Modélisation du système de recommandation](#modelisation)  
    4.1 [Recommandation collaborative](#modelisation-colabo)  
    4.2 [Recommandation basée sur le contenu](#modelisation-content)  
    4.3 [Recommandations pour les nouveaux utilisateurs](#modelisation-new)  
    4.4 [Recommandation collaborative (bibliothèque Surprise)](#modelisation-surprise)   
5. [Mise à jour des bases de données](#updatedata)  
    5.1 [Gestion des articles](#update-articles)  
    5.2 [Gestion des utilisateurs](#update-users)   
6. [Synthèse](#synthese)  

# Importation des librairies Python spécialisées :

In [ ]:
print("Test du serveur Jupyter")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import pickle
import os, zipfile, time
import warnings
import random
from sklearn.model_selection import train_test_split as sk_train_test_split
from scipy.sparse import csr_matrix, save_npz

In [ ]:
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import KNNBasic
from surprise import CoClustering
from surprise import BaselineOnly
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, ndcg_at_k

from sklearn.metrics.pairwise import cosine_similarity

# Définitions des fonctions :
- Les fonctions ont été regroupées par catégories pour faciliter leur utilisation afin de rendre le code plus lisible et plus facile à maintenir. Les fonctions sont donc centralisées ci-après.

## Cleaning

In [ ]:
def convert_columns_to_integer(data, column_names):
    """
    Convert specified columns in a DataFrame to integer type.

    Parameters
    ----------
    data (pd.DataFrame): The DataFrame containing the data.
    column_names (list): A list of column names to be converted to integer type.

    Returns
    -------
    pd.DataFrame: A new DataFrame with the specified columns converted to integer type.
    """
    # Make a copy of the DataFrame to avoid modifying the original
    data_copy = data.copy()

    # Convert specified columns to integer type
    for column_name in column_names:
        data_copy[column_name] = data_copy[column_name].astype(int)

    return data_copy

In [ ]:
def convert_columns_to_categorical(data, column_names):
    """
    Convert specified columns in a DataFrame to categorical type.

    Parameters
    ----------
    data (pd.DataFrame): The DataFrame containing the data.
    column_names (list): A list of column names to be converted to categorical type.

    Returns
    -------
    pd.DataFrame: A new DataFrame with the specified columns converted to categorical type.
    """
    # Make a copy of the DataFrame to avoid modifying the original
    data_copy = data.copy()

    # Convert specified columns to categorical type
    for column_name in column_names:
        data_copy[column_name] = data_copy[column_name].astype('category')

    return data_copy

## Evaluation

In [ ]:
def add_evaluation_scores(model_name, predictions, df):
    """
    Add evaluation scores to the DataFrame.

    Parameters
    ----------
    model_name: str
        Name of the model.
    predictions: list
        List of model predictions.
    df: pd.DataFrame
        DataFrame to which evaluation results are appended.

    Returns
    -------
    pd.DataFrame
        Updated DataFrame with new evaluation scores.
    """
    from surprise import accuracy

    rmse = accuracy.rmse(predictions, verbose=False)
    mse = accuracy.mse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)

    # Create a new DataFrame with the results
    new_row = pd.DataFrame([{ 'model': model_name, 'rmse': rmse, 'mse': mse, 'mae': mae }])

    # Use pd.concat() instead of append()
    df = pd.concat([df, new_row], ignore_index=True)
    
    return df


In [ ]:
import pandas as pd

def evaluate_model_and_append_to_df(model, csr_train, csr_test, model_name, df_cb_evaluation_results):
    """
    Evaluate a model's performance and append the results to a DataFrame.

    Parameters
    -----------
    model: object
        The model to be evaluated.
    csr_train: sparse matrix
        The training data in compressed sparse row format.
    csr_test: sparse matrix
        The test data in compressed sparse row format.
    model_name: str
        The name of the model.
    df_cb_evaluation_results: pd.DataFrame
        The DataFrame to which evaluation results will be appended.

    Returns
    --------
    df_cb_evaluation_results
    """
    precision = round(precision_at_k(model, csr_train, csr_test, K=5, show_progress=True, num_threads=1), 5)
    map_score = round(mean_average_precision_at_k(model, csr_train, csr_test, K=5, show_progress=True, num_threads=1), 5)
    ndcg = round(ndcg_at_k(model, csr_train, csr_test, K=5, show_progress=True, num_threads=1), 5)

    # Create a new DataFrame with the results
    new_row = pd.DataFrame([{
        'model': model_name,
        'precision': precision,
        'map': map_score,
        'ndcg': ndcg,
    }])

    # Use pd.concat() instead of append() for better compatibility
    df_cb_evaluation_results = pd.concat([df_cb_evaluation_results, new_row], ignore_index=True)

    return df_cb_evaluation_results

In [ ]:
def calculate_user_recommendation_similarity(model, df, csr_train, num_recommendations=5, max_users=None):
    """
    Calculate user-article recommendation similarity using embeddings and cosine similarity.

    Parameters
    -----------
    model: object
        The collaborative filtering model used for making recommendations.
    df: pandas.DataFrame
        The DataFrame containing user-article interaction data.
    csr_train: scipy.sparse.csr.csr_matrix
        The training set in CSR format.
    num_recommendations: int, optional
        The number of article recommendations to generate for each user (default is 5).
    max_users: int, optional
        Maximum number of users to consider for recommendation calculation (default is None).

    Returns
    --------
    float
        Overall mean cosine similarity between viewed articles and recommended articles.
    pandas.DataFrame
        DataFrame with user recommendations and their cosine similarity scores.
    """
    # For each user, get the IDs of the viewed articles
    user_articles = df.groupby('user_id')['click_article_id'].apply(list).reset_index(name='viewed_articles')

    # Remove users with only one article viewed
    user_articles = user_articles[user_articles.viewed_articles.map(len) > 1]

    # Crop selection if specified
    if max_users is not None:
        user_articles = user_articles[:max_users]

    # For each user, compute the mean embedding vectors of the viewed articles
    user_articles['viewed_mean_embedding'] = user_articles.apply(
        lambda x: articles_embeddings[np.array(x[1])].mean(axis=0), axis=1
    )

    # For each user, make recommendations
    user_ids = user_articles['user_id'].values
    recommended_article_ids, scores = model.recommend(user_ids, csr_train[user_ids], N=num_recommendations, filter_already_liked_items=True)
    user_articles['recommended_articles'] = list(recommended_article_ids)

    # For each user, compute the mean embedding vectors of the recommended articles
    user_articles['recommended_mean_embedding'] = user_articles['recommended_articles'].apply(
        lambda x: articles_embeddings[np.array(x)].mean(axis=0)
    )

    # For each user, compute the cosine similarity between the mean_embedding and the pred_mean_embedding
    user_articles['cos_similarity'] = user_articles.apply(
        lambda x: cosine_similarity(x['viewed_mean_embedding'].reshape(1, -1), x['recommended_mean_embedding'].reshape(1, -1))[0][0],
        axis=1
    )

    # Reset the index column
    user_articles = user_articles.set_index('user_id')

    # Compute & return the overall mean cosine similarity
    return user_articles['cos_similarity'].mean(), user_articles

## Filter

In [ ]:
def get_columns_by_type(df, dtype):
    """
    This function takes a Pandas dataframe and a data type as input,
    and returns the names of the columns that have the specified data type.
    """
    return df.select_dtypes(include=[dtype]).columns.tolist()

In [ ]:
def get_most_recent_article_per_user(df):
    """
    Get the most recent article clicked by each user.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame containing user-article interaction data.

    Returns
    -------
    pd.DataFrame
        A DataFrame with 'user_id' and 'article_id' columns representing the most recent article clicked by each user.
    """
    most_recent_article_per_user = df.groupby('user_id').click_timestamp.max().reset_index()
    most_recent_article_per_user = most_recent_article_per_user.merge(df, on=['user_id', 'click_timestamp'], how='left')
    most_recent_article_per_user = most_recent_article_per_user[['user_id', 'click_article_id']]
    most_recent_article_per_user.columns = ['user_id', 'article_id']

    return most_recent_article_per_user

In [ ]:
def get_most_recent_articles_for_user(df, user_id, n=1):
    """
    Get the n most recent articles clicked by a specific user.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame containing user-article interaction data.
    user_id : int
        The ID of the user for whom recent articles are retrieved.
    n : int, optional
        The number of most recent articles to retrieve (default is 1).

    Returns
    -------
    pd.Series
        A Series with the most recent article IDs for the user.
    """
    user_articles = df[df.user_id == user_id]
    most_recent_articles = user_articles.nlargest(n, 'click_timestamp')['click_article_id']

    return most_recent_articles

In [ ]:
def get_unseen_articles(article_metadata, clicks, n=5):
    """
    Get n random article IDs from article metadata that are not included in the clicks DataFrame.

    Parameters
    ----------
    article_metadata: pd.DataFrame
        DataFrame containing article metadata.
    clicks: pd.DataFrame
        DataFrame containing user-article interaction data.
    n: int, optional
        The number of random unseen article IDs to return (default is 5).

    Returns
    -------
    pd.Series
        A Series with n random article IDs not present in the clicks DataFrame.
    """
    seen_articles = clicks['click_article_id'].unique()
    unseen_articles = article_metadata[~article_metadata['article_id'].isin(seen_articles)]['article_id']

    if len(unseen_articles) < n:
        return unseen_articles
    else:
        return pd.Series(random.sample(unseen_articles.tolist(), n))

## Graph

In [ ]:
def boxplot_columns(data, columns, figsize=(15, 6), x=""):
    """
    Create a set of boxplots for specified columns in a DataFrame.

    Parameters
    ----------
    data (pd.DataFrame): The DataFrame containing the data.
    columns (list): A list of column names to create boxplots for.
    figsize (tuple, optional): A tuple specifying the figure size (width, height). Default is (15, 6).
    x (str, optional): The column name for the x-axis, if needed. Default is an empty string.

    Returns
    -------
    None
    """
    n_rows = (len(columns) + 2) // 3  # Calculate the number of rows required for subplots
    plt.figure(figsize=figsize)

    for index, col in enumerate(columns, start=1):
        plt.subplot(n_rows, 3, index)
        sns.boxplot(x=data[col])
        plt.xlabel(x)
        plt.title(f"Boxplot for {col}")

    plt.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_histogram(df, quantitative_column, bins=10, x_title="Value"):
    """
    Create a histogram chart for a quantitative column in a Pandas DataFrame using Plotly.

    Parameters
    ----------
    df (pd.DataFrame): The DataFrame containing the quantitative column.
    quantitative_column (str): The name of the quantitative column in the DataFrame.
    bins (int): The number of bins for the histogram.
    x_title (str): The title for the x-axis.

    Returns
    -------
    None
    """
    # Vérifier si la colonne existe
    if quantitative_column not in df.columns:
        raise KeyError(f"La colonne '{quantitative_column}' n'existe pas dans le DataFrame.")

    title = f"Histogram for {quantitative_column}"

    # Create the histogram chart using Plotly
    fig = go.Figure(data=[go.Histogram(x=df[quantitative_column], nbinsx=bins, histnorm='percent')])
    fig.update_layout(
        title=title,
        xaxis_title=x_title,
        yaxis_title="%",
        width=800,
        height=600
    )
    fig.show()



In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_timestamp_chart(df, timestamp_column, title="Timestamp Chart"):
    """
    Crée un graphique chronologique des timestamps en regroupant par mois et année.

    Parameters
    ----------
    df (pd.DataFrame): Le DataFrame contenant la colonne de timestamps.
    timestamp_column (str): Le nom de la colonne contenant les timestamps.
    title (str): Le titre du graphique.

    Returns
    -------
    None
    """
    # Vérifier si le timestamp est en secondes ou millisecondes
    if df[timestamp_column].max() < 10**10:  # Probablement en secondes
        df['formatted_timestamp'] = pd.to_datetime(df[timestamp_column], unit='s')
    else:  # Probablement en millisecondes
        df['formatted_timestamp'] = pd.to_datetime(df[timestamp_column], unit='ms')

    # Extraire l'année et le mois
    df['year_month'] = df['formatted_timestamp'].dt.to_period("M").astype(str)  # Format YYYY-MM

    # Regrouper et compter les occurrences par mois
    monthly_counts = df.groupby('year_month').size().reset_index(name='count')

    # Calculer les pourcentages
    total_count = len(df)
    monthly_counts['percentage'] = (monthly_counts['count'] / total_count) * 100

    # Trier chronologiquement
    monthly_counts = monthly_counts.sort_values(by='year_month')

    # Créer un graphique Plotly
    fig = go.Figure(data=[go.Bar(
        x=monthly_counts['year_month'],
        y=monthly_counts['percentage'],
    )])

    fig.update_layout(
        title=title,
        xaxis_title='Date (Mois Année)',
        yaxis_title='Pourcentage',
        width=900,
        height=500
    )

    fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_histogram(df, quantitative_column, bins=10, x_title="Value"):
    """
    Create a histogram chart for a quantitative column in a Pandas DataFrame using Plotly.

    Parameters
    ----------
    df (pd.DataFrame): The DataFrame containing the quantitative column.
    quantitative_column (str): The name of the quantitative column in the DataFrame.
    bins (int): The number of bins for the histogram.
    x_title (str): The title for the x-axis.

    Returns
    -------
    None
    """
    # Vérifier si la colonne existe
    if quantitative_column not in df.columns:
        raise KeyError(f"La colonne '{quantitative_column}' n'existe pas dans le DataFrame.")

    title = f"Histogram for {quantitative_column}"

    # Créer l'histogramme avec Plotly
    fig = go.Figure(data=[go.Histogram(x=df[quantitative_column], nbinsx=bins, histnorm='percent')])
    fig.update_layout(
        title=title,
        xaxis_title=x_title,
        yaxis_title="%",
        width=800,
        height=600
    )
    fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_timestamp_chart_day_format(df, timestamp_column, title="Timestamp Chart"):
    """
    Create a chronological chart of a timestamp column in a Pandas DataFrame using Plotly.

    Parameters
    ----------
    df (pd.DataFrame): The DataFrame containing the timestamp column.
    timestamp_column (str): The name of the timestamp column in the DataFrame.
    title (str): The title for the chart.

    Returns
    -------
    None
    """

    # Vérifier si les valeurs sont déjà en datetime
    if not pd.api.types.is_datetime64_any_dtype(df[timestamp_column]):
        # Vérifier si les valeurs sont en secondes ou en millisecondes
        if df[timestamp_column].max() < 10**10:  # Probablement en secondes
            df['formatted_timestamp'] = pd.to_datetime(df[timestamp_column], unit='s')
        else:  # Probablement en millisecondes
            df['formatted_timestamp'] = pd.to_datetime(df[timestamp_column], unit='ms')
    else:
        df['formatted_timestamp'] = df[timestamp_column]  # Si déjà datetime, on l'utilise tel quel

    # Extraire uniquement la partie jour/mois/année sous forme de texte
    df['day_month_year'] = df['formatted_timestamp'].dt.strftime('%d %B %Y')

    # Compter les occurrences de chaque jour
    daily_counts = df['day_month_year'].value_counts().sort_index()

    # Création du graphique avec Plotly
    fig = go.Figure(data=[go.Bar(
        x=daily_counts.index,
        y=daily_counts.values,
    )])

    fig.update_layout(
        title=title,
        xaxis_title='Date (Day Month Year)',
        yaxis_title='Count',
        width=800,
        height=600
    )

    fig.show()


In [ ]:
def categorical_column_distribution_chart(df, category_column, top=0, without_dups_col="", x_title="Category", title=""):
    """
    Creates a bar chart of the top X classes of a categorical column in a Pandas DataFrame using Plotly.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the categorical column.
    category_column : str
        The name of the categorical column in the DataFrame.
    top : int
        The number of top categories to display in the chart. If 0, all categories will be displayed.
    without_dups_col : list
        The name of the column to drop duplicates on.

    Returns:
    --------
    None
    """
    if without_dups_col:
        df = df.drop_duplicates(without_dups_col + [category_column])

    # Count the frequency of each class in the categorical column and calculate the percentages
    category_counts = df[category_column].value_counts(normalize=True) * 100
    top_categories = category_counts

    title = title if title else f"Bar chart of {category_column} categories"

    if top > 0:
        # Select the top `top` classes
        top_categories = category_counts[:top]
        title = f"Bar chart of top {top} {category_column} categories"

    # Create the bar chart using Plotly
    fig = go.Figure([go.Bar(x=top_categories.index, y=top_categories.values)])
    fig.update_layout(
        title=title,
        xaxis_title=x_title,
        yaxis_title="Percentage",
        width=800,
        height=600
    )
    fig.update_traces(text=top_categories.values.round(2), textposition='outside')
    fig.show()

In [ ]:
def plot_user_article_distribution(data):
    """
    Create a bar chart displaying the distribution of users by the number of articles they have read using Plotly.

    Parameters
    -----------
    data : pandas.DataFrame
        The DataFrame containing user-article interaction data.

    Returns
    --------
    None
    """
    # Group the data by 'user_id' and calculate the count of articles read by each user
    user_article_counts = data[['user_id','click_article_id']].copy()
    user_article_counts = pd.DataFrame(user_article_counts.groupby('user_id').size(), columns=['num_articles'])
    user_article_counts = pd.DataFrame(user_article_counts.groupby('num_articles').size(), columns=['num_users'])

    user_article_counts["num_users_percent"] = 100 / data['user_id'].nunique() * user_article_counts

    # Create a bar chart using Plotly
    fig = px.bar(user_article_counts, x=user_article_counts.index, y='num_users_percent',
                 labels={'num_users_percent': 'Utilisateurs (%)', 'index': 'Nombre d\'articles lus'},
                 title='Distribution des utilisateurs par nombre d\'articles lus')

    fig.show()

In [ ]:
def plot_article_per_user_distribution(data):
    """
    Create a distribution plot displaying the percentage of articles read and the number of users using Plotly.

    Parameters:
    -----------
    data : pandas.DataFrame
        The DataFrame containing user-article interaction data.

    Returns:
    --------
    None
    """
    # Group the data by 'user_id' and calculate the count of articles read by each user
    user_article_counts = data.groupby('user_id')['click_article_id'].count()

    # Calculate the percentages for articles read
    total_articles = user_article_counts.sum()
    article_percentages = (user_article_counts / total_articles) * 100

    # Create a histogram chart using Plotly
    fig = px.histogram(user_article_counts, x=user_article_counts, y=article_percentages, nbins=50,
                      labels={'x': 'Nombre de lectures'},
                      title='Répartition des articles par nombre de lectures')
    fig.update_layout(yaxis_title='Nombre d\'article lus (%)' )
    fig.show()

In [ ]:
def plot_sns_histogram(df, quantitative_column, bins=10, x_title="Value"):
    """
    Create a histogram chart for a quantitative column in a Pandas DataFrame using Seaborn.

    Parameters
    ----------
    df (pd.DataFrame): The DataFrame containing the quantitative column.
    quantitative_column (str): The name of the quantitative column in the DataFrame.
    bins (int): The number of bins for the histogram.
    x_title (str): The title for the x-axis.

    Returns
    -------
    None
    """
    title = f"Histogram for {quantitative_column}"

    plt.figure(figsize=(8, 6))
    sns.histplot(data=df, x=quantitative_column, stat='percent', bins=bins, kde=True)
    plt.xlabel(x_title)
    plt.ylabel("%")
    plt.title(title)
    plt.show()

## Modify

In [ ]:
def add_mean_cos_similarity(df, model_name, mean_cos_similarity):
    """
    Add the mean_cos_similarity value for a specific model to the DataFrame.

    Parameters
    -----------
    df: pandas.DataFrame
        The DataFrame to which the value will be added.
    model_name: str
        The name of the model.
    mean_cos_similarity: float
        The mean cosine similarity value to be added.

    Returns
    --------
    None
    """
    # Check if the model already exists in the DataFrame
    if model_name in df['model'].values:
        # Update the 'mean_cos_similarity' value for the specific model
        df.loc[df['model'] == model_name, 'mean_cos_similarity'] = mean_cos_similarity
    else:
        # Add a new row for the model with 'mean_cos_similarity'
        df = df.append({'model': model_name, 'mean_cos_similarity': mean_cos_similarity}, ignore_index=True)

In [ ]:
def calculate_mean_embedding(article_ids):
    """
    Calculate the mean embedding for a list of article IDs.

    Parameters
    ----------
    article_ids : list
        A list of article IDs.

    Returns
    -------
    numpy.ndarray
        The mean embedding vector for the provided article IDs.
    """
    mean_embedding = lambda x: articles_embeddings[x].mean(axis=0)
    return mean_embedding(article_ids)

In [ ]:
def add_new_article(df_articles_metadata, new_article_data):
    """
    Add a new article to the df_articles_metadata DataFrame.

    Parameters
    ----------
    df_articles_metadata: pd.DataFrame
        The DataFrame containing article metadata.
    new_article_data: dict
        A dictionary containing the data for the new article, including category_id,
        created_at_ts, publisher_id, and words_count.

    Returns
    -------
    pd.DataFrame
        The updated df_articles_metadata DataFrame with the new article.
    """
    # Calculate the next available article_id by incrementing the maximum article_id in the DataFrame
    next_article_id = df_articles_metadata['article_id'].max() + 1 if not df_articles_metadata.empty else 1

    # Create a new row with the calculated article_id and provided data
    new_article_row = {
        'article_id': next_article_id,
        'category_id': new_article_data['category_id'],
        'created_at_ts': new_article_data['created_at_ts'],
        'publisher_id': new_article_data['publisher_id'],
        'words_count': new_article_data['words_count']
    }

    # Use pd.concat() instead of append()
    new_row_df = pd.DataFrame([new_article_row])
    df_articles_metadata = pd.concat([df_articles_metadata, new_row_df], ignore_index=True)

    return df_articles_metadata

## Preprocessing

In [ ]:
def calculate_popularity_indicator(df_merged):
    """
    Calculate the popularity indicator for user-article interactions.

    Parameters
    -----------
    df_merged : pd.DataFrame
        The DataFrame containing user-article interaction data.

    Returns
    --------
    pd.DataFrame
        A DataFrame with 'user_id', 'click_article_id', 'user_article_clicks', 'user_total_clicks',
        and 'popularity_indicator' columns.
    """
    # Group the data by user_id and click_article_id to count user-article clicks
    user_article_clicks = df_merged.groupby(['user_id', 'click_article_id'])['click_article_id'].count()

    # Group the data by user_id to count total clicks
    user_total_clicks = df_merged.groupby('user_id')['click_article_id'].count()

    # Create DataFrames from Series
    df_user_article_clicks = user_article_clicks.reset_index(name='user_article_clicks')
    df_user_total_clicks = user_total_clicks.reset_index(name='user_total_clicks')

    # Merge the two DataFrames based on 'user_id'
    df_popularity_ind = pd.merge(df_user_article_clicks, df_user_total_clicks, on='user_id')

    # Calculate the 'popularity_indicator' based on the merged DataFrames
    df_popularity_ind['popularity_indicator'] = df_popularity_ind['user_article_clicks'] / df_popularity_ind['user_total_clicks']

    return df_popularity_ind

In [ ]:
def calculate_category_popularity(df_merged):
    """
    Calculate the category-specific popularity indicator for user-article interactions.

    Parameters
    -----------
    df_merged : pd.DataFrame
        The DataFrame containing user-article interaction data.

    Returns
    --------
    pd.DataFrame
        A DataFrame with 'user_id', 'category_id', 'user_category_clicks', 'user_total_clicks',
        and 'category_popularity' columns.
    """
    # Group the data by user_id and category_id and count the number of category clicks
    user_category_clicks = df_merged.groupby(['user_id', 'category_id'])['category_id'].count()

    # Group the data by user_id and calculate the total clicks
    user_total_clicks = df_merged.groupby('user_id')['click_article_id'].count()

    # Create DataFrames from the Series
    df_user_category_clicks = user_category_clicks.reset_index(name='user_category_clicks')
    df_user_total_clicks = user_total_clicks.reset_index(name='user_total_clicks')

    # Merge the two DataFrames based on 'user_id'
    df_popularity_ind = pd.merge(df_user_category_clicks, df_user_total_clicks, on='user_id')

    # Calculate the 'category_popularity' as the ratio of category-specific clicks to total clicks by the user
    df_popularity_ind['category_popularity'] = df_popularity_ind['user_category_clicks'] / df_popularity_ind['user_total_clicks']

    return df_popularity_ind

## Recommendation

In [ ]:
def recommend_top_articles(model, user_id, num_recommendations=10):
    """
    Recommend the top articles for a given user using a collaborative filtering model.

    Parameters:
    -----------
    model: object
        The collaborative filtering model used for making recommendations.
    user_id: int
        The ID of the user for whom recommendations are generated.
    num_recommendations: int, optional
        The number of article recommendations to return (default is 10).

    Returns:
    --------
    pd.DataFrame
        A DataFrame with 'article_id' and 'score' columns representing the top article recommendations.
    """
    articles_read = df_article_popularity[df_article_popularity.user_id == user_id].click_article_id.unique()
    print("Total articles viewed by the user {}: {}".format(user_id, len(articles_read)))
    articles_not_read = df_articles_metadata[~df_articles_metadata.article_id.isin(articles_read)].article_id.unique()
    print("Total articles not viewed by the user {}: {}".format(user_id, len(articles_not_read)))

    articles_not_read_with_score = [[article_id, model.predict(user_id, article_id).est] for article_id in articles_not_read]
    articles_not_read_with_score_sorted = sorted(articles_not_read_with_score, key=lambda x: x[1], reverse=True)

    top_recommendations = articles_not_read_with_score_sorted[:num_recommendations]
    recommendation_df = pd.DataFrame(top_recommendations, columns=['article_id', 'score'])

    return recommendation_df

In [ ]:
def get_top_popular_articles(df, num_recommendations=10):
    """
    Get the top popular articles based on the popularity indicator.

    Parameters
    -----------
    df: pd.DataFrame
        DataFrame containing user-article interaction data, including 'click_article_id' and 'popularity_indicator' columns.
    num_recommendations: int, optional
        The number of top popular articles to recommend (default is 10).

    Returns
    --------
    pd.DataFrame
        A DataFrame with the top popular articles and their popularity scores.
    """
    # Group the data by 'article_id' and calculate the sum of popularity indicators
    df_popular_articles = df.groupby(by=['click_article_id'])['popularity_indicator'].sum().sort_values(ascending=False).reset_index()

    # Normalize the popularity scores to be between 0 and 1
    df_popular_articles['score'] = df_popular_articles['popularity_indicator'] / df_popular_articles['popularity_indicator'].max()

    # Print a message and create a DataFrame with the top popular articles
    print(f'Top {num_recommendations} popular articles:')
    recommendation = df_popular_articles[['click_article_id', 'score']].head(num_recommendations).to_dict('records')
    recommendation_df = pd.DataFrame(recommendation)
    recommendation_df = recommendation_df.rename(columns={'click_article_id': 'article_id'})

    return recommendation_df


In [ ]:
def recommend_articles_to_users(model, user_ids, csr_matrix, num_recommendations=5):
    """
    Recommend a list of articles to multiple users using a collaborative filtering model.

    Parameters
    -----------
    model: object
        The collaborative filtering model used for making recommendations.
    user_ids: list
        A list of user IDs for whom recommendations are generated.
    csr_matrix: csr_matrix
        The user-article interaction matrix.
    num_recommendations: int, optional
        The number of article recommendations to return for each user (default is 5).

    Returns
    --------
    None
    """
    article_ids, scores = model.recommend(user_ids, csr_matrix[user_ids], N=num_recommendations, filter_already_liked_items=True)

    for i, user_id in enumerate(user_ids):
        print(f"\nRecommended articles for the user {user_id}\n")

        for article_id, score in zip(article_ids[i], scores[i]):
            print(f"article_id: {article_id:10} \t score: {score}")

In [ ]:
# Using mean cosine similarity
def get_content_based_recommendations(user_id, num_recommendations=5, user_history_length=None):
    """
    Get content based article recommendations for a user based on their recent article views.

    Parameters
    ----------
    user_id: int
        The ID of the user for whom recommendations are generated.
    num_recommendations: int, optional
        The number of article recommendations to return (default is 5).
    user_history_length: int, optional
        The number of recent article views to consider (default is None, which includes all views).

    Returns
    -------
    list
        A list of top article recommendations for the user.
    """
    if user_history_length is not None:
        # Get the specified number of most recent article views for the user
        #recent_article_views = df_train[df_train.user_id == user_id]['article_id'].iloc[-user_history_length:]
        recent_article_views = get_most_recent_articles_for_user(df_train, user_id, n=user_history_length)
    else:
        # Get all article views for the user if user_history_length is not specified
        recent_article_views = df_train[df_train.user_id == user_id]['article_id']

    if len(recent_article_views) < 1:
        return []

    # Calculate the mean embedding of the recent article views
    mean_recent_article_views = calculate_mean_embedding(recent_article_views.values)

    # Create a copy of article embeddings and subtract the mean embedding for recent views
    embeddings = articles_embeddings.copy()
    mean_recent_views = mean_recent_article_views
    embeddings[recent_article_views.values] = -mean_recent_views

    # Calculate cosine similarity between modified embeddings and the mean recent views
    cosine = cosine_similarity(embeddings, mean_recent_views.reshape(1, -1))

    # Get top article recommendations based on cosine similarity
    recommendations = get_top_recommendations(cosine, num_recommendations)

    return recommendations


def get_top_recommendations(cosine, num_recommendations=5):
    """
    Get the top cosine similarity recommendations.

    Parameters
    ----------
    cosine: pd.Series
        A Series containing cosine similarity values.
    num_recommendations: int, optional
        The number of recommendations to return (default is 5).

    Returns
    -------
    pd.DataFrame
        A DataFrame with 'article_id' and 'cos_similarity' columns for the top recommendations.
    """
    recommendations = pd.DataFrame(cosine, columns=['cos_similarity'])
    recommendations = recommendations.sort_values('cos_similarity', ascending=False)[:num_recommendations]
    recommendations = recommendations.reset_index()
    recommendations = recommendations.rename(columns={'index': 'article_id'})

    return recommendations

In [ ]:
import pandas as pd

def recommend_content_based_articles(df, user_id, num_recommendations=5, user_history_length=1):
    """
    Generate recommendations for a user based on their recent article views and unseen articles.

    Parameters
    ----------
    df: pd.DataFrame
        The DataFrame containing user-article interaction data.
    user_id: int
        The user for whom recommendations are generated.
    num_recommendations: int, optional
        The total number of recommendations to return (default is 5).
    user_history_length: int, optional
        The number of most recent article views to consider (default is 1).

    Returns
    -------
    pd.DataFrame
        A DataFrame of article IDs recommended for the user.
    """
    # Get most recent articles viewed by a user
    if user_history_length is not None:
        recent_article_views = get_most_recent_articles_for_user(df_clicks, user_id, n=user_history_length)
    else:
        recent_article_views = df[df.user_id == user_id]['article_id']

    # Get the specified number of unseen articles
    n_unseen = num_recommendations % 4
    unseen_articles = get_unseen_articles(df_articles_metadata, df_clicks, n=n_unseen)

    # Initialize a dataframe for recommendations
    recommended_articles = pd.DataFrame(columns=['article_id', 'cos_similarity_score'])

    # For each recent article, find similar articles
    for article_id in recent_article_views:
        similar_articles = get_similar_articles(article_id, num_recommendations)
        # Use pd.concat() instead of append()
        recommended_articles = pd.concat([recommended_articles, similar_articles], ignore_index=True)

    # Randomly select a subset of similar articles
    random_selection = recommended_articles.sample(n=num_recommendations - n_unseen, replace=True)
    
    # Loop to add unseen article IDs to the DataFrame
    unseen_df = pd.DataFrame({'article_id': unseen_articles, 'cos_similarity_score': 0})
    random_selection = pd.concat([random_selection, unseen_df], ignore_index=True)
    
    return random_selection


def get_similar_articles(article_id, n=5):
    """
    Get the most similar articles to a given article.

    Parameters
    ----------
    article_id : int
        The article ID for which to find similar articles.
    n : int, optional
        Number of similar articles to retrieve (default is 5).

    Returns
    -------
    pd.DataFrame
        A DataFrame with columns 'article_id' and 'cos_similarity_score' representing similar articles.
    """
    # Filter embeddings to remove unseen articles
    filtered_embeddings = filter_embeddings(articles_embeddings)

    # Get the index of the article in the embeddings
    idx = filtered_embeddings.index.get_loc(article_id)

    # Calculate the cosine similarity between the article and all other articles
    cos_similarity = cosine_similarity(filtered_embeddings, filtered_embeddings)

    # Get the similarity scores of the article with other articles
    similarity_scores = list(enumerate(cos_similarity[idx]))

    # Sort the similarity scores in descending order
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the n most similar articles
    similarity_scores = similarity_scores[1:n + 1]

    # Get the indices of the similar articles
    articles_index = [i[0] for i in similarity_scores]

    # Get the article IDs and their corresponding similarity scores
    article_ids, cos_similarity_scores = filtered_embeddings.iloc[articles_index].index, [i[1] for i in similarity_scores]

    # Create a DataFrame with similar articles and their similarity scores
    df_recommended_articles = pd.DataFrame({
        'article_id': article_ids,
        'cos_similarity_score': cos_similarity_scores
    })

    return df_recommended_articles


def filter_embeddings(embeddings):
    """
    Filter embeddings to only include those corresponding to viewed articles.

    Parameters
    ----------
    embeddings: pd.DataFrame
        DataFrame containing embeddings for all articles.

    Returns
    -------
    pd.DataFrame
        Filtered embeddings containing only those corresponding to viewed articles.
    """
    embeddings = pd.DataFrame(embeddings)
    viewed_articles = df_clicks['click_article_id'].value_counts().index

    return embeddings.iloc[viewed_articles]

# Config

In [ ]:
warnings.filterwarnings('ignore')

# 1. Présentation générale du jeu de données <a name="presentation"></a>

Les données utilisées pour ce projet peuvent être téléchargées à partir du lien suivant : https://www.kaggle.com/datasets/gspmoreira/news-portal-user-interactions-by-globocom#clicks_sample.csv
Le jeu de données contient un échantillon des interactions des utilisateurs (consultations de pages) sur le portail d'actualités G1 du 1er au 16 octobre 2017. Il comprend environ 3 millions de clics répartis dans plus d'un million de sessions, impliquant 314 000 utilisateurs ayant consulté plus de 46 000 articles d'actualités différents pendant cette période.

Il se compose de trois fichiers/dossiers :

* `clicks` - Un dossier avec des fichiers CSV (un par heure) contenant les interactions des utilisateurs lors de leurs sessions sur le portail d'actualités.  
* `articles_metadata.csv` - Un fichier CSV contenant des informations sur les métadonnées de tous les articles publiés (364 047 au total).  
* `articles_embeddings.pickle` - Un fichier au format Pickle (Python 3) contenant une matrice NumPy avec les représentations d'incorporation du contenu des articles (vecteurs de 250 dimensions). Ces vecteurs ont été créés en utilisant le module ACR de CHAMELEON, en se basant sur le texte et les métadonnées des articles, et couvrent les 364 047 articles publiés. Regarder https://arxiv.org/abs/1808.00076 pour plus détails.
* `cliks_sample.csv` Un fichier CSV contenant un échantillon des interactions des utilisateurs (consultations de pages) sur le portail d'actualités. Il comprend environ 724955 de clics répartis, impliquant 314 000 utilisateurs sur la période du 23 décembre. Ce sample est utilisé pour des raisons de performance et à la même structure que les fichiers individuels dans le dossier clicks. Cela permettra de determiner la qualité de notre modèle sur un échantillon de données.


## 1.1. Chargement des données <a name="presentation-data"></a>

## 1.2. Inspection des données <a name="presentation-inspection"></a>

### Dossier des clicks

In [ ]:
import os
import pandas as pd

# Définition du bon chemin (relatif au notebook)
clicks_dir = "data/clicks/clicks"

# Vérifier que le dossier existe
if not os.path.exists(clicks_dir):
    raise FileNotFoundError(f"Le dossier {clicks_dir} n'existe pas.")

# Liste tous les fichiers CSV dans le dossier clicks
csv_files = [f for f in os.listdir(clicks_dir) if f.endswith(".csv")]

# Trier les fichiers pour garantir l'ordre de lecture correct
csv_files.sort()

# Initialiser une liste pour stocker les DataFrames
datalist = []

# Boucle pour lire tous les fichiers CSV
for file in csv_files:
    file_path = os.path.join(clicks_dir, file)  
    df_tmp = pd.read_csv(file_path)
    datalist.append(df_tmp)

# Fusionner tous les fichiers en un seul DataFrame
df_dossier_clicks = pd.concat(datalist, ignore_index=True)

# ---- ANALYSE DES DONNÉES ----

# Nombre total de fichiers CSV chargés
num_files = len(csv_files)

# Nombre total de sessions uniques
num_sessions = df_dossier_clicks['session_id'].nunique()

# Nombre total de clics (chaque ligne correspond à un clic)
num_clicks = len(df_dossier_clicks)

# Nombre d'articles uniques ayant été cliqués
num_articles_clicked = df_dossier_clicks['click_article_id'].nunique()

# Nombre d'utilisateurs uniques
num_users = df_dossier_clicks['user_id'].nunique()

# Obtenir un résumé statistique pour toutes les colonnes numériques
stats = df_dossier_clicks.describe()

# Afficher les résultats
print(f"📂 Nombre de fichiers CSV chargés : {num_files}")
print(f"👥 Nombre total d'utilisateurs uniques : {num_users}")
print(f"🖥️ Nombre total de sessions uniques : {num_sessions}")
print(f"🖱️ Nombre total de clics : {num_clicks}")
print(f"📰 Nombre total d'articles cliqués : {num_articles_clicked}")
print("\n📊 Statistiques descriptives :")
print(stats)


### **🔍 Observations sur les Données Clicks**
L'analyse des **385 fichiers CSV** donne les résultats suivants :

#### **📊 Synthèse Générale**
- **📂 Nombre de fichiers CSV chargés :** `385`
- **👥 Nombre total d'utilisateurs uniques :** `322,897`
- **🖥️ Nombre total de sessions uniques :** `1,048,594`
- **🖱️ Nombre total de clics enregistrés :** `2,988,181`
- **📰 Nombre total d'articles cliqués :** `46,033`

#### **🔍 Observations par colonne**
1. **`session_id` et `user_id`**
   - Il y a **322,897 utilisateurs uniques** et **1,048,594 sessions uniques**.
   - Certains utilisateurs ont effectué plusieurs sessions.
   - La session la plus fréquente a été observée **124 fois**.

2. **`session_size`**
   - La taille de session varie sur **72 valeurs distinctes**, ce qui signifie que certaines sessions sont très courtes (2 clics) tandis que d'autres sont beaucoup plus longues.

3. **`click_article_id`**
   - **46,033 articles différents** ont été cliqués.
   - L'article le plus cliqué **(ID : 160974)** a été sélectionné **37,213 fois**.

4. **`click_environment`**
   - Seulement **3 environnements différents** sont recensés.
   - **L'environnement le plus fréquent représente 2,904,478 clics**, ce qui montre un fort biais vers un environnement dominant.

5. **`click_deviceGroup` et `click_os`**
   - **5 types d’appareils différents** sont utilisés, mais **1 seul appareil représente 1,823,162 clics**.
   - **8 systèmes d’exploitation différents** sont détectés, mais **1 seul (ID : 17) est majoritaire avec 1,738,138 clics**.

6. **`click_country` et `click_region`**
   - Les clics proviennent de **11 pays différents**, mais **1 pays représente 2,852,406 clics**, ce qui montre une forte concentration géographique.
   - **28 régions** sont identifiées, avec une région dominante **(804,985 clics)**.

7. **`click_referrer_type`**
   - **7 types de référents** sont utilisés.
   - **Le référent le plus courant (ID : 2) est responsable de 1,602,601 clics**, ce qui montre que la majorité des visiteurs proviennent d’une source principale.

---

### **📌 Décision pour la Suite**
Compte tenu du **volume de données élevé (~3 millions de lignes)**, nous allons travailler sur un **échantillon plus léger : `clicks_sample.csv`** pour faciliter les manipulations et analyses tout en conservant une représentativité des tendances globales.

🚀 **Les prochaines étapes se baseront donc sur `clicks_sample.csv` pour des raisons de volumétrie et de performance.**

### Fichier sample des clicks pour l'analyse et les recommandations

In [ ]:
import pandas as pd
import pickle

# Charger les métadonnées des articles
df_articles_metadata = pd.read_csv('data/articles_metadata.csv')

# Charger les embeddings des articles
with open('data/articles_embeddings.pickle', 'rb') as file:
    articles_embeddings = pickle.load(file)

# Charger les données de clicks depuis un seul fichier
df_clicks = pd.read_csv('data/clicks_sample.csv')

# Ajouter une colonne "hour" qui n'existait pas avant
df_clicks['hour'] = pd.to_datetime(df_clicks['click_timestamp'], unit='ms').dt.hour

print(f"✅ Chargement réussi : {len(df_clicks)} lignes de clics importées depuis clicks_sample.csv")


### Fichier Métadonnées

In [ ]:
df_articles_metadata.info()

Ce jeu de données est composé de 364 047 lignes décrites par 5 variables

Variable|Description|
|--------|-----------|
|__article_id__ | identifiant d'un article|
|__category_id__ | identifiant d'une catégorie|
| __created_at_ts__ | horodatage (en millisecondes) lorsque l'article a été créé|
| __publisher_id__ | identifiant de l'éditeur|
| __words_count__ | nombre de mots dans un article|

In [ ]:
# Afficher les 5 premières lignes
df_articles_metadata.head()

In [ ]:
# Afficher les 5 dernières lignes pour s'assurer que les données sont chargées correctement
df_articles_metadata.tail()

In [ ]:
# Description des données e.g. moyenne, médiane, min, max, etc.
df_articles_metadata.describe()

Vérifions s'il existe des doublons.

In [ ]:
df_articles_metadata.duplicated(subset=['article_id']).sum()

In [ ]:
print("Le nombre total des articles :", df_articles_metadata['article_id'].nunique())
print("Le nombre total des catégories :", df_articles_metadata['category_id'].nunique())
print("Le nombre total des éditeurs :", df_articles_metadata['publisher_id'].nunique())

### Les clics

In [ ]:
df_clicks.info()

In [ ]:
# nombre des lignes et des colonnes
df_clicks.shape

Voici un **workflow simple** expliquant le fonctionnement du fichier CSV :

---

### **Workflow : Analyse des clics des utilisateurs sur des articles**
📌 **Objectif** : Suivre l'activité des utilisateurs sur une plateforme en analysant leurs sessions et leurs clics sur des articles.

#### 1️⃣ **Début d'une session utilisateur**  
- Un utilisateur **(user_id)** démarre une session **(session_id)** à un instant donné **(session_start)**.  
- Une session contient un certain **nombre d'articles vus** **(session_size)**.

#### 2️⃣ **Enregistrement des clics**  
- Chaque ligne représente **un clic** effectué par un utilisateur sur un article **(click_article_id)**.  
- Le **timestamp du clic (click_timestamp)** indique le moment précis où l'article a été consulté.  
- Plusieurs clics peuvent être enregistrés pour une même session.

#### 3️⃣ **Contexte des clics**  
- **click_environment** : Plateforme ou environnement où le clic s'est produit (ex: mobile, desktop).  
- **click_deviceGroup** : Type d’appareil utilisé (ex: smartphone, PC, tablette).  
- **click_os** : Système d’exploitation utilisé (ex: Windows, iOS, Android).  
- **click_country** et **click_region** : Localisation de l'utilisateur.  
- **click_referrer_type** : Type de site depuis lequel l'utilisateur est arrivé (ex: moteur de recherche, réseau social).

#### 4️⃣ **Fin de session**  
- Une session regroupe **tous les clics enregistrés pour un même session_id**.  
- La **durée de la session (hour)** peut être calculée en prenant la différence entre le premier et le dernier **click_timestamp**.

---

### **Exemple illustré**
#### **Session de l'utilisateur 0**  
👤 **User_id** = 0  
🆔 **Session_id** = 1506825423271737  
🕒 **Session_start** = 1506825423000  
📄 **Session_size** = 2  

| Click # | Click Article ID | Click Timestamp |
|---------|-----------------|----------------|
| 1 | 157541 | 1506826828020 |
| 2 | 68866  | 1506826858020 |

🔹 L'utilisateur **a cliqué sur 2 articles** pendant cette session.  
🔹 Il a cliqué sur le premier article à **1506826828020** et le second à **1506826858020**.

---

### **Conclusion**
Ce fichier permet de **suivre l'activité des utilisateurs**, d'identifier les articles les plus consultés et d'analyser le comportement des utilisateurs en fonction de leur appareil, localisation et source de trafic.

|Variable|Description|
|--------|-----------|
|__user_id__ | identifiant d'utilisateur|
|__session_id__ | identifiant d'une session|
| __session_start__ | date de début de la session|
| __session_size__ | nombre d'articles vus pendant la session|
| __click_article_id__ | identifiant d'article sur lequel on a cliqué|
| __click_timestamp__ | Horodatage (timestamp) du moment où un clic sur un article a eu lieu|
| __click_environment__ | environnement ou plate-forme où l'événement de clic s'est produit|
| __click_deviceGroup__ | type d'appareil ou de plate-forme qu'un utilisateur utilise|
| __click_os__ | identifiant du système d'exploitation|
| __click_country__ | pays de l'utilisateur|
| __click_region__ | région de l'utilisateur|
| __click_referrer_type__ | type de site de provenance|
| __hour__ | durée de la session en heures|

`click_environment`:
1. Facebook
2. Mobile App
3. Mobile Pages
4. Web  

`click_deviceGroup`:
1. Tablet  
2. TV  
4. Mobile
5. Computer
3. blank  

`click_os`:  
1. Other  
2. iOS  
3. Android  
4. Windows Phone  
5. Windows Mobile  
6. Windows  
7. Mac OS X  
8. Mac OS  
9. Samsung  
10. FireHbbTV  
11. ATV OS X  
12. tvOS  
13. Chrome OS  
14. Debian  
15. Symbian OS  
16. BlackBerry OS  
17. Firefox OS  
18. Android  
19. Brew MP  
20. Chromecast  
21. webOS  
22. Gentoo  
23. Solaris

### **⚠️ Attention aux timestamps !**
L’analyse des données montre que la colonne **`click_timestamp`** contient des valeurs élevées, comme **1506961009961**.  

**💡 Explication du timestamp :**
- En informatique, un timestamp est une valeur numérique qui représente un **instant précis dans le temps**.
- Il est généralement exprimé en **secondes ou en millisecondes** depuis le **1er janvier 1970 à minuit UTC** (appelé l’**Epoch Unix**).

**🔍 Comment vérifier ?**
- Si les valeurs du timestamp sont **trop grandes** (comme `1506961009961`), elles sont probablement en **millisecondes**.
- Un timestamp en **secondes** aurait une valeur plus courte, typiquement autour de **10 chiffres** (`1506961000`).

**🛠️ Que faire ?**
👉 **Convertir les timestamps en dates lisibles** en fonction de leur unité :

#### **1️⃣ Si le timestamp est en secondes :**
```python
df_dossier_clicks['click_timestamp'] = pd.to_datetime(df_dossier_clicks['click_timestamp'], unit='s')
```

#### **2️⃣ Si le timestamp est en millisecondes :**
```python
df_dossier_clicks['click_timestamp'] = pd.to_datetime(df_dossier_clicks['click_timestamp'], unit='ms')
```

**📌 Pourquoi c'est important ?**
- Une mauvaise conversion peut **décaler** toutes les dates d'environ **31 000 ans** 🕰️ !
- **Avant toute analyse temporelle (filtrage par date, tendances, etc.), il est crucial de bien interpréter l’unité du timestamp.**

🚀 **Pour la suite du projet, nous allons valider si les timestamps sont en secondes ou millisecondes et les convertir correctement.**

### transformer certaines colonnes d'un DataFrame df_clicks en types de données spécifiques, à savoir des entiers et des catégories pour prétraiter les données.

In [ ]:
# int_cols est une liste des colonnes à convertir en entier
int_cols = ['user_id', 'session_id', 'session_start', 'session_size', 'click_article_id', 'click_timestamp', 'hour']

# cat_cols est une liste des colonnes à convertir en catégorie
cat_cols = ['click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']

# Convertir les colonnes en entier et en catégorie dans le DataFrame df_clicks
df_clicks = convert_columns_to_integer(df_clicks, int_cols)
df_clicks = convert_columns_to_categorical(df_clicks, cat_cols)

In [ ]:
# Mapping des valeurs catégorielles pour les colonnes click_environment, click_deviceGroup et click_os afin de rendre les données plus lisibles.

environment_mapping = {1: "Facebook", 2: "Mobile App", 3: "Mobile Pages", 4: "Web"}
os_mapping = {
    1: "Other",
    2: "iOS",
    3: "Android",
    4: "Windows Phone",
    5: "Windows Mobile",
    6: "Windows",
    7: "Mac OS X",
    8: "Mac OS",
    9: "Samsung",
    10: "FireHbbTV",
    11: "ATV OS X",
    12: "tvOS",
    13: "Chrome OS",
    14: "Debian",
    15: "Symbian OS",
    16: "BlackBerry OS",
    17: "Firefox OS",
    18: "Android",
    19: "Brew MP",
    20: "Chromecast",
    21: "webOS",
    22: "Gentoo",
    23: "Solaris"
}

device_mapping = {
    1: "Tablet",
    2: "TV",
    3: "Mobile",
    4: "Computer",
    5: "blank"
}


# Appliquer le mapping aux colonnes correspondantes
df_clicks['click_environment'] = df_clicks['click_environment'].map(environment_mapping)
df_clicks['click_deviceGroup'] = df_clicks['click_deviceGroup'].map(device_mapping)
df_clicks['click_os'] = df_clicks['click_os'].map(os_mapping)

In [ ]:
# Afficher les 5 premières lignes
df_clicks.head()

In [ ]:
df_clicks.tail()

In [ ]:
df_clicks.describe()

Vérifions s'il existe des doublons.

In [ ]:
df_clicks.duplicated().sum()

# 2. Analyse des données <a name="analyse"></a>

## 2.1. Valeurs aberrantes <a name="analyse-outliers"></a>

Pour visualiser les outliers représentons les valeurs des variables quantitatives sous forme des boîtes à moustaches.

In [ ]:
boxplot_columns(df_articles_metadata, ['created_at_ts', 'words_count'], figsize=(25, 6))

In [ ]:
cols = ['session_start', 'session_size', 'click_timestamp', 'hour']
boxplot_columns(df_clicks, cols, figsize=(25, 10))

Il n'y a pas de valeurs impossibles pour ces variables.

## 2.2. Analyse univariée <a name="analyse-uni"></a>

### Les catégories d'article

In [ ]:
categorical_column_distribution_chart(df_articles_metadata, category_column='category_id', x_title='Catégorie', title="Répartition des catégories par article")

### Observations :

- En abscisse, on a les **catégories** d'articles, identifiées par leur ID (`category_id`).
- En ordonnée, on a le **pourcentage** de chaque catégorie par rapport au total des articles.
- Pour rappel, il y a **364 047 articles** dans le jeu de données répartis dans **461 catégories différentes**.
- La catégorie **281** est la plus représentée dans le jeu de données, dépassant **3,5 %** du total.
- La majorité des catégories ont une proportion inférieure à **1 %**, suggérant une longue traîne dans la distribution.
- Certaines catégories au-delà de l’ID **400** montrent des pics notables, mais elles restent relativement rares.
- La distribution semble très déséquilibrée, avec une dominance marquée par quelques catégories, tandis que beaucoup d'autres ont une faible présence.

### Suggestions pour l'analyse :
- Vérifier si certaines catégories peuvent être regroupées selon des critères communs pour réduire l’hétérogénéité.
- Explorer les catégories peu représentées pour évaluer leur pertinence ou leur niche dans le système de recommandation.
- Étudier les corrélations potentielles entre la **taille des articles** (via `words_count`) et leur **catégorie**.


### Date de création d'articles :
- Sert à identifier les périodes de publication des articles pour évaluer la fraîcheur des données.

In [ ]:
# Afficher la distribution de la date de création des articles
plot_timestamp_chart(df_articles_metadata, 'created_at_ts', title="Date de création d'articles")

### **Observations sur la distribution des dates de création des articles** 📊
- Le graphique montre la distribution des dates de création des articles qui s'est étendue sur une période de plusieurs années, d'environ de 2013 à 2018.  

### Nombre de mots dans des articles :
- Permet d'évaluer la longueur des articles et leur complexité
- Pour la recommandation, cela peut être utilisé pour évaluer la diversité des articles.

In [ ]:
# Afficher la distribution du nombre de mots dans les articles
plot_histogram(df_articles_metadata, 'words_count', bins=500, x_title="word")

Le graphique montre la distribution de nombre de mots dans les articles.  
D'après le graphique, la majorité des articles, soit 60 %, contiennent moins de 200 mots.

### Date de début de la session

In [ ]:
import pandas as pd

# Charger les données
df_clicks = pd.read_csv("data/clicks_sample.csv")

# Vérifier les valeurs brutes
print("Valeurs brutes de session_start :")
print(df_clicks['session_start'].describe())

# Vérifier si le timestamp est en secondes ou millisecondes
if df_clicks['session_start'].max() < 10**10:  # Probablement en secondes
    df_clicks['session_start'] = pd.to_datetime(df_clicks['session_start'], unit='s')
else:  # Probablement en millisecondes
    df_clicks['session_start'] = pd.to_datetime(df_clicks['session_start'], unit='ms')

# Vérification après conversion
print("Valeurs converties :")
print(df_clicks['session_start'].describe())

# Afficher la période correcte
print("La période des données s'étend du {} au {}".format(
    df_clicks['session_start'].min().strftime('%Y-%m-%d %H:%M:%S'),
    df_clicks['session_start'].max().strftime('%Y-%m-%d %H:%M:%S')
))


In [ ]:
# Afficher la distribution de la date de début de la session. L'objectif est de comprendre la période de consultation des articles. "plot_timestamp_chart_day_format" est une fonction personnalisée pour afficher les graphiques de distribution de la date.
plot_timestamp_chart_day_format(df_clicks, 'session_start', title="Date de début de la session")

### Observations sur la distribution des dates de début de session :
- Le graphique montre la distribution des dates de début de session, qui s'étend sur une période de 23 décembre 2017. Cette période sur une journée est due à l'échantillon de données utilisé qui est un sous-ensemble des clics et donc restreint dans le temps.

### Date du clic

In [ ]:
plot_timestamp_chart_day_format(df_clicks, 'click_timestamp', title="Date du clic")

### Durée de la session

In [ ]:
print(df_clicks.columns)  # Vérifiez que 'hour' est bien dans la liste des colonnes


In [ ]:
# Avant d'appeler la fonction, s'assurer que la colonne 'hour' existe
if 'hour' not in df_clicks.columns:
    df_clicks['hour'] = df_clicks['session_start'].dt.hour

# Maintenant on peut appeler la fonction
plot_histogram(df_clicks, 'hour', bins=20, x_title="Hour of the day")

 Le graphique indique que la plupart des sessions commencent entre 2h et 3h du matin, ce qui pourrait être une anomalie ou une tendance nocturne à analyser

### Session size

In [ ]:
plot_histogram(df_clicks, 'session_size', bins=60, x_title="hour")

### **📊 Analyse du graphique sur la taille des sessions**
Le graphique représente la distribution de la taille des sessions (`session_size`), c'est-à-dire **le nombre de clics (articles consultés) par session**.

---

### **🔍 Observations**
1. **La majorité des sessions sont courtes**  
   - La **grande majorité des sessions contiennent moins de 10 articles**.
   - Un **pic très marqué** est visible pour les petites sessions (2 à 4 articles).

2. **Diminution rapide**  
   - Plus la taille de la session augmente, **moins on observe de sessions**.
   - Il y a **quelques sessions exceptionnelles avec un nombre élevé de clics** (autour de **15 et 20 articles**), mais elles sont très rares.

3. **Possible biais utilisateur**  
   - Le comportement des utilisateurs semble **être tourné vers des sessions courtes**.
   - Cela peut être lié à une **navigation rapide**, une **faible fidélisation** ou une **consultation ponctuelle d'articles précis**.


🚀 **Cette analyse suggère que les utilisateurs consultent peu d’articles par session, ce qui peut être un facteur clé dans l’amélioration de l’expérience utilisateur et de la recommandation de contenu.**

### Environnement du clic

In [ ]:
categorical_column_distribution_chart(df_clicks, category_column='click_environment', without_dups_col=['session_id'], x_title='Environnement', title="Type d'environnement utilisé dans différentes sessions")

### **📊 Analyse du graphique sur l’environnement utilisé**
Ce graphique représente la distribution des **types d’environnement** utilisés par les utilisateurs lors des sessions.

---

### **🔍 Observations**
1. **Un environnement ultra-dominant**  
   - L’environnement **"4"** est **massivement majoritaire**, représentant **95,63% des sessions**.
   - Cela suggère que la quasi-totalité des utilisateurs passent par **une seule plateforme** (probablement **le Web**).

2. **Présence d’autres environnements, mais en très faible proportion**  
   - **L’environnement "2"** représente **3,81%** des sessions.
   - **Les environnements "1" et "0.5"** sont **quasiment inexistants**, avec **0,56% et 0,5%** des sessions.

3. **Hypothèse sur la signification des valeurs**  
   - Si **4 = Web**, alors la navigation **via le Web domine largement l’usage**.
   - **D’autres environnements** pourraient correspondre à des **applications mobiles, TV, ou autres plateformes**, mais ils sont **marginaux**.

🚀 **Ce graphique confirme que l’essentiel de la navigation se fait via un seul type d’environnement, ce qui peut avoir un impact sur les recommandations et l’optimisation de la plateforme.**

### Type d'appareil utilisé

In [ ]:
categorical_column_distribution_chart(df_clicks, category_column='click_deviceGroup', without_dups_col=['session_id'], x_title='Appareil', title="Divers appareils utilisés lors des sessions")

### **📊 Analyse du graphique sur les appareils utilisés**
Ce graphique représente la répartition des différents **types d’appareils** utilisés par les utilisateurs lors de leurs sessions.

---

### **🔍 Observations**
1. **Deux types d’appareils dominent l’usage**  
   - L’appareil **"3"** est le plus utilisé, représentant **56,72% des sessions**.
   - L’appareil **"1"** suit avec **37,62% des sessions**.

2. **Un troisième type d’appareil marginal**  
   - L’appareil **"4"** est **nettement moins utilisé**, avec seulement **5,66% des sessions**.

3. **Interprétation possible**  
   - Si **"3" représente les tablettes**, alors celles-ci sont **le mode de navigation principal**.
   - **"1" pourrait correspondre aux smartphones**, expliquant pourquoi une part significative des sessions y est enregistrée.
   - **"4" pourrait être les ordinateurs**, ce qui indiquerait **un usage relativement faible du desktop**.


🚀 **L’analyse montre une préférence pour la navigation mobile (tablettes et smartphones), ce qui peut être crucial pour l’optimisation du contenu et des recommandations.**

### Système d'exploitation utilisé

In [ ]:
categorical_column_distribution_chart(df_clicks, category_column='click_os', without_dups_col=['session_id'], x_title='OS', title="Divers OS utilisés lors des sessions")

### **📊 Analyse mise à jour du graphique sur les systèmes d’exploitation (OS) utilisés**
Grâce aux informations supplémentaires, nous pouvons maintenant **associer les valeurs affichées sur le graphique aux véritables OS correspondants**.

---

### **🔍 Observations mises à jour**
1. **Trois OS dominent l’usage**  
   - **Firefox OS (37.48%)** est le système le plus utilisé par les utilisateurs.
   - **Android (33.8%)** arrive en **deuxième position**, confirmant une forte utilisation mobile.
   - **Chromecast (24.05%)** représente également une part significative, indiquant que **de nombreux utilisateurs consomment du contenu via un appareil de streaming**.

2. **Présence marginale d’autres OS**  
   - **iOS (3.25%)** est étonnamment peu représenté, contrairement à ce que l’on pourrait attendre.
   - **D’autres systèmes comme Windows, Mac OS X, et Linux sont presque absents**, ce qui montre que **les ordinateurs classiques ne sont pas les supports privilégiés**.

3. **Enjeux pour l’optimisation des recommandations**  
   - **Une forte proportion d’utilisateurs sur Firefox OS et Android** implique qu’**une grande partie de l’audience provient d’appareils mobiles et TV connectées**.
   - **L’adaptation des recommandations et du design au mobile et aux plateformes de streaming est essentielle**.
   - **L’absence notable d’OS desktop** (Windows, Mac OS) indique que **les utilisateurs privilégient des sessions courtes et mobiles plutôt qu’une navigation approfondie sur ordinateur**.

🚀 **Avec cette analyse, il est clair que l’accent doit être mis sur l’optimisation mobile et sur l’expérience utilisateur des plateformes de streaming, car elles constituent la majorité des usages.**

### Pays depuis le quel le clic a été réalisé

In [ ]:
categorical_column_distribution_chart(df_clicks, category_column='click_country', without_dups_col=['session_id'], x_title='Code de pays', title="Localisation des clics par pays")

Le graphique montre d'où proviennent les clics en fonction des pays.  
Près de 95 % de ces clics ont été effectués depuis le pays associé au code 1, probablement le Brésil.

### Région de l'utilisateur

In [ ]:
categorical_column_distribution_chart(df_clicks, category_column='click_region', without_dups_col=['session_id'], x_title='Code de région', title="Localisation des clics par région")

### **📊 Analyse du graphique sur la localisation des utilisateurs par région**
Ce graphique représente la répartition des **clics en fonction des régions**, permettant d’identifier les zones géographiques les plus actives.

---

### **🔍 Observations**
1. **Deux régions dominent largement**  
   - La région **25** représente **26,94% des sessions**, ce qui en fait la zone la plus active en termes de clics.
   - La région **20** suit avec **17,66% des sessions**, confirmant son importance dans l’engagement des utilisateurs.

2. **Quelques autres régions avec une activité modérée**  
   - Les régions **15 (7,34%)** et **16 (6,36%)** montrent une activité significative mais bien inférieure aux deux principales.
   - La région **10 (5,23%)** a également un certain volume d’interactions.

3. **De nombreuses régions peu représentées**  
   - Un grand nombre de codes de régions affichent une part inférieure à **5%**, ce qui montre une **forte concentration des clics sur quelques zones spécifiques**.
   - Certaines régions sont presque inexistantes en termes d’interactions.

4. **Interprétation des résultats**  
   - **Les recommandations et contenus personnalisés pourraient être optimisés pour les régions les plus actives** (25 et 20).
   - **Une analyse plus approfondie est nécessaire** pour comprendre pourquoi certaines régions ont une faible participation : manque de couverture, faible population ou préférences de navigation différentes.


🚀 **Ces observations montrent l’importance d’adapter les recommandations aux préférences régionales et de comprendre pourquoi certaines régions sont moins actives.**

### Type de site de provenance

In [ ]:
categorical_column_distribution_chart(df_clicks, category_column='click_referrer_type', without_dups_col=['session_id'], x_title='site', title="Type de site de provenance")

### **📊 Analyse du graphique sur le type de site de provenance**
Ce graphique illustre la **répartition des sessions en fonction du type de site de provenance**, c'est-à-dire d’où proviennent les utilisateurs avant de cliquer sur un article.

---

### **🔍 Observations**
1. **Deux types de sites dominent clairement**  
   - Le **type 2** est le plus fréquent, représentant **49,49%** des sessions.
   - Le **type 1** suit de près avec **41,52%**, ce qui en fait également une source majeure de trafic.

2. **Autres sources de provenance moins utilisées**  
   - Le **type 5** représente **4,05%** des sessions, ce qui est relativement faible par rapport aux deux premiers.
   - Les types **7 (2,41%)**, **6 (1,52%)** et **4 (1,01%)** ont une très faible contribution.

3. **Interprétation des résultats**  
   - **Les recommandations et stratégies d’acquisition devraient être adaptées aux types de sites majoritaires (1 et 2).**  
   - **Une analyse plus approfondie des sites peu utilisés pourrait être utile** pour comprendre s’ils présentent un potentiel inexploité.

🚀 **Ces observations mettent en évidence la nécessité d'optimiser le contenu pour les sources de trafic les plus fréquentes, tout en explorant de nouvelles opportunités d’acquisition.**

## 2.3. Analyse multivariée <a name="analyse-multi"></a>

### Répartition des utilisateurs par nombre d'articles lus

In [ ]:
plot_user_article_distribution(df_clicks)

### **📊 Analyse de la répartition des utilisateurs par nombre d'articles lus**
Ce graphique illustre la **distribution du nombre d'articles lus par les utilisateurs**.

---

### **🔍 Observations**
1. **Une grande majorité des utilisateurs lisent très peu d'articles**  
   - Plus de **60% des utilisateurs lisent un seul article**.
   - Environ **20% des utilisateurs lisent entre 2 et 3 articles**.
   - Au-delà de 4 articles, le nombre d’utilisateurs chute drastiquement.

2. **Lecture intensive très rare**  
   - Très peu d’utilisateurs lisent plus de **10 articles**.
   - **L’audience est donc plutôt passive**, avec une faible consommation de contenu.

3. **Interprétation des résultats**  
   - **Il est crucial de capter l’attention dès le premier article pour encourager la lecture d’autres contenus.**  
   - **Des recommandations d’articles plus engageantes** ou une personnalisation du contenu pourraient inciter les utilisateurs à lire davantage.


🚀 **Ces observations soulignent l'importance d'améliorer la pertinence des recommandations pour augmenter la durée des sessions et la fidélité des utilisateurs.**

### Répartition des articles par nombre de clics

In [ ]:
plot_article_per_user_distribution(df_clicks)

### **📊 Analyse de la répartition des articles par nombre de lectures**
Ce graphique représente la **distribution du nombre de lectures par article**.

---

### **🔍 Observations**
1. **Une très forte concentration des clics sur un faible nombre d'articles**  
   - Près de **50% des articles n'ont été lus qu'une seule fois**.
   - Environ **30% des articles ont reçu entre 2 et 3 lectures**.
   - Très peu d'articles dépassent **10 lectures**.

2. **Un phénomène de long tail marqué**  
   - La majorité des articles sont peu consultés, tandis qu'une minorité génère un nombre significatif de clics.
   - Cela souligne l'importance d’un **algorithme de recommandation efficace** pour exposer plus d'articles aux utilisateurs.

3. **Opportunités d'amélioration**  
   - **Optimiser la diversité des recommandations** pour éviter une surexposition des mêmes articles.
   - **Augmenter la découvrabilité des articles moins lus** via des recommandations contextuelles ou personnalisées.


💡 **Conclusion :**  
➡️ **Ce déséquilibre dans la distribution des lectures indique un fort biais de consommation des articles.**  
➡️ **Les recommandations devront aider à équilibrer la visibilité des contenus pour améliorer l'engagement global.**

In [ ]:
print("Le nombre total des utilisateurs :", df_clicks['user_id'].nunique())
print("Le nombre total des sessions :", df_clicks['session_id'].nunique())

# 3. Préparation du jeu de données d'étude <a name="preparation">

## 3.1.  Fusion des données <a name="preparation-fusion">

Nous allons fusionner df_clicks et df_articles_metadata, en se basant sur les colonnes `click_article_id` et `article_id`, ce qui donne comme résultat un DataFrame fusionné (`df_merged`).

In [ ]:
df_merged = pd.merge(df_clicks, df_articles_metadata, how='left', left_on='click_article_id', right_on='article_id')
df_merged.info()

In [ ]:
df_merged.head()

In [ ]:
df_merged.tail()

## 3.2. Élaboration d'un indicateur de popularité pour des articles <a name="preparation-score">

Pour créer un indicateur de popularité pour des articles j'ai décidé de me baser su le nombre de clics d'un utilisateur sur un article divisé par le nombre total de clics de l'utilisateur.  
Cette approche est simple à mettre en œuvre, car elle repose sur des informations de clics disponibles et ne nécessite pas de données complexes ou de modèles sophistiqués.  

Le score calculé est spécifique à chaque utilisateur, ce qui permet de personnaliser les recommandations en fonction du comportement individuel de l'utilisateur. Les utilisateurs ayant des habitudes de clics distinctes recevront des recommandations adaptées à leurs préférences.   

Cependant, il convient de noter que cette approche a aussi des limites, notamment en ce qui concerne la diversité des recommandations et la gestion des biais potentiels, mais elle constitue un point de départ solide pour de nombreux systèmes de recommandation, en particulier dans les cas où les données sont limitées ou lorsqu'une simplicité d'implémentation est privilégiée.

In [ ]:
df_article_popularity = calculate_popularity_indicator(df_merged)
df_article_popularity.info()

In [ ]:
df_article_popularity.describe()

In [ ]:
df_article_popularity = df_article_popularity[['user_id', 'click_article_id', 'popularity_indicator']]
#df_article_popularity = df_article_popularity.sample(frac=0.1, random_state=42)
df_article_popularity.shape

In [ ]:
plot_sns_histogram(df_article_popularity, 'popularity_indicator', x_title='score')

## 3.3. Séparation du jeu de donnée <a name="preparation-split">

### Pour les modèles de la librarie 'Surprise'La préparation de données pour un système de recommandation utilisant la bibliothèque Surprise se déroule en trois étapes :

- Création d’un objet Reader : La première étape consiste à créer un objet Reader, qui définit l’échelle des notes (ratings). Dans ce cas, l’échelle est fixée de 0 à 0.5, conformément aux observations montrant qu’aucun utilisateur ne se concentre sur un seul article.
- Chargement du jeu de données : Une fois l’objet Reader créé, on charge le DataFrame df_article_popularity dans la bibliothèque Surprise à l’aide de la méthode Dataset.load_from_df(). Cette méthode applique l’échelle de notation définie dans le Reader.
- Division de l’ensemble de données : Enfin, on sépare le jeu de données en deux parties : un ensemble d’apprentissage (trainset) et un ensemble de test (testset). 20 % des données sont affectées au test, tandis que les 80 % restants servent à l’apprentissage du modèle.

In [ ]:
# Permet de définir l'échelle des notations attendues dans l'ensemble de données
max_popularity = df_article_popularity['popularity_indicator'].max()
print(f"Max popularity_indicator previously observed: {max_popularity}")

unique_articles_per_user = df_merged.groupby('user_id')['click_article_id'].nunique().describe()
print(unique_articles_per_user)

In [ ]:
# Create a "Reader" object for defining the rating scale (minimum and maximum ratings)
reader = Reader(rating_scale=(0, 0.5))

# Load the dataset from the DataFrame df_article_popularity using the defined reader
dataset = Dataset.load_from_df(df_article_popularity, reader)

# Split the dataset into a training set and a test set, with 20% of the data in the test set
trainset, testset = train_test_split(dataset, test_size=.20)

### Pour les modèles de la librairie 'Implicit'

Ici, nous allons simplement diviser le DataFrame "df_article_popularity" en deux parties, où 80 % des données sont réservées pour l'apprentissage du modèle (df_train) et 20 % sont réservées pour tester la performance du modèle (df_test).  
Ensuite, nous allons créer des matrices CSR à partir des données d'apprentissage (df_train) et des données de test (df_test). Ces matrices sont essentielles pour des modèles de recommandation, comme les systèmes de filtrage collaboratif, ils utilisent ces matrices pour générer des recommandations personnalisées aux utilisateurs.

In [ ]:
df_train, df_test = sk_train_test_split(df_article_popularity, train_size=0.8, random_state=42)

dimentions = (max(df_train.user_id.max(), df_test.user_id.max()) + 1, max(df_train.click_article_id.max(), df_test.click_article_id.max()) + 1)

csr_train = csr_matrix((df_train['popularity_indicator'], (df_train['user_id'], df_train['click_article_id'])), dimentions)
csr_test = csr_matrix((df_test['popularity_indicator'], (df_test['user_id'], df_test['click_article_id'])), dimentions)

In [ ]:
# Création de la matrice CSR pour la popularité des articles
csr_article_popularity = csr_matrix((df_article_popularity['popularity_indicator'], (df_article_popularity['user_id'], df_article_popularity['click_article_id'])), dimentions)

# Définition du chemin du dossier "models"
models_dir = os.path.join(os.getcwd(), "models")

# Vérifier si le dossier existe, sinon le créer
os.makedirs(models_dir, exist_ok=True)

# Sauvegarde de la matrice CSR de popularité des articles
save_npz(os.path.join(models_dir, "csr_article_popularity.npz"), csr_article_popularity)

print(f"Matrice CSR de popularité des articles sauvegardée avec succès dans : {models_dir}")

# 4. Modélisation du système de recommandation <a name="modelisation">

Nous allons désormais explorer deux approches pour développer notre système de recommandation :

* Filtrage collaboratif (Collaborative Filtering) : Cette méthode repose sur l'idée que les utilisateurs ayant des préférences similaires ont tendance à apprécier les mêmes articles.

* Filtrage basé sur le contenu (Content-Based Filtering) : Cette méthode se base sur l'idée que des articles similaires ont tendance à plaire aux mêmes utilisateurs."

Pour évaluer la performance de nos modèles de recommandation, nous utiliserons quatre métriques différentes :

1. La __précision__ : Cette mesure évalue quelle proportion des articles recommandés est effectivement pertinente pour un utilisateur donné. Plus précisément, elle calcule le nombre d'articles pertinents parmi les recommandations, divisé par le nombre total d'articles recommandés.

2. Le __Mean Average Precision__ (MAP) : Le MAP évalue la qualité du classement des articles recommandés. Il calcule la précision moyenne sur toutes les positions de la liste de recommandations. Une caractéristique importante du MAP est qu'il tient compte des articles pertinents qui ne sont pas recommandés, les incluant dans le calcul de la précision moyenne. Cela permet de mesurer la qualité globale du classement.

3. Le __Normalized Discounted Cumulative Gain__ (NDCG) : Le NDCG évalue la qualité du classement des articles recommandés en considérant l'ordre de recommandation. Il prend en compte à la fois la pertinence de chaque article recommandé et sa position dans la liste de recommandations. Cela permet d'évaluer la qualité du classement en fonction de l'importance de la position de chaque article.

Ces trois métriques sont couramment utilisées pour évaluer les systèmes de recommandation, car elles évaluent à la fois la pertinence des recommandations et l'ordre dans lequel elles sont présentées.

En outre, compte tenu de la disponibilité des embeddings pour l'ensemble de nos articles, nous envisageons de créer une métrique basée sur leur similarité. Cette nouvelle métrique consisterait à comparer l'embedding moyen des articles lus avec l'embedding moyen des articles recommandés. Cela nous permettrait d'évaluer à quel point les sujets des articles recommandés correspondent à ceux qui ont été lus. Cependant, il est important de noter que cette métrique ne nous indiquera pas si les articles recommandés ont effectivement été lus par l'utilisateur, mais plutôt à quel point le contenu des articles recommandés est similaire à celui des articles lus.

In [ ]:
df_evaluation_results = pd.DataFrame(columns=('model', 'rmse', 'mse', 'mae'))
df_cb_evaluation_results = pd.DataFrame(columns=('model', 'mean_cos_similarity', 'precision', 'map', 'ndcg'))

## 4.1. Recommandation collaborative <a name="modelisation-colabo">

Dans cette partie la bibliothèque Implicit est utilisée pour la recommandation basée sur les préférences implicites des utilisateurs. Contrairement à la bibliothèque Surprise qui se concentre sur les données explicites (notes ou évaluations), Implicit se concentre sur les interactions implicites telles que les clics, les likes, les commentaires et les lectures. Cette librairie a été spécifiquement conçue pour les systèmes de recommandation utilisant des données implicites concernant le comportement des utilisateurs. Notre dataset ne contenant aucune note explicite sur les articles, nous allons utiliser cette librairie.

### AlternatingLeastSquares

Alternating Least Squares (ALS) est un algorithme couramment utilisé en filtrage collaboratif pour créer des systèmes de recommandation personnalisés. Cet algorithme vise à modéliser les préférences des utilisateurs et les caractéristiques des articles en optimisant une fonction de coût spécifique.

In [ ]:
als_model = AlternatingLeastSquares(
    factors=32,
    regularization=0.05,
    iterations=50,
    alpha=40
)

als_model.fit(csr_train)

Utilisons notre modèle pour suggérer des articles à certains utilisateurs.

In [ ]:
# Trouver la plage d'indices valides dans csr_test
num_users = csr_test.shape[0]  # Nombre d'utilisateurs dans le CSR (Customer-User Matrix)
print(f"Nombre d'utilisateurs disponibles : {num_users}")

# # Filtrer les user_ids valides
# valid_user_ids = [user_id for user_id in user_ids if user_id < num_users]
# print(f"User IDs valides : {valid_user_ids}")
# 
# # Appliquer la fonction de recommandation uniquement aux utilisateurs valides
# if valid_user_ids:
#     recommend_articles_to_users(als_model, valid_user_ids, csr_test)
# else:
#     print("Aucun user ID valide trouvé")


In [ ]:
user_ids = [15, 706]
recommend_articles_to_users(als_model, user_ids, csr_test)

__Evaluation du modèle AlternatingLeastSquares__

Evaluons à quel point les recommandations d'articles sont similaires aux articles déjà consultés par les utilisateurs. Pour cela on utilise des embeddings d'articles et la similarité cosinus pour mesurer cette similarité.

In [ ]:
mcs_score, df_mcs = calculate_user_recommendation_similarity(als_model, df_test, csr_train)

In [ ]:
df_mcs.head()

In [ ]:
df_mcs.describe()

In [ ]:
print(f'mean cosine similarity: {mcs_score}')

In [ ]:
%history -g | grep "als_model"


In [ ]:
als_model # Verifier si le modèle existe déja

In [ ]:
df_cb_evaluation_results = evaluate_model_and_append_to_df(als_model, csr_train, csr_test, 'AlternatingLeastSquares', df_cb_evaluation_results)

In [ ]:
add_mean_cos_similarity(df_cb_evaluation_results, 'AlternatingLeastSquares', mcs_score)
df_cb_evaluation_results

### LogisticMatrixFactorization

Logistic Matrix Factorization (LMF) est une approche de filtrage collaboratif qui incorpore des éléments de modélisation probabiliste pour estimer les probabilités d'interaction entre les utilisateurs et les articles. Elle est utilisée pour générer des recommandations personnalisées en se basant sur des données implicites et des facteurs latents.

In [ ]:
lmf_model = LogisticMatrixFactorization(
    factors=32,
    learning_rate=0.05,
    regularization=0.05,
    iterations=50,
)

lmf_model.fit(csr_train)

Utilisons notre modèle pour suggérer des articles à certains utilisateurs.

In [ ]:
user_ids = [15, 706]
recommend_articles_to_users(lmf_model, user_ids, csr_test)

__Evaluation du modèle LogisticMatrixFactorization__

Evaluons à quel point les recommandations d'articles sont similaires aux articles déjà consultés par les utilisateurs. Pour cela on utilise des embeddings d'articles et la similarité cosinus pour mesurer cette similarité.

In [ ]:
mcs_score, df_mcs = calculate_user_recommendation_similarity(lmf_model, df_test, csr_train)
df_mcs.head()

In [ ]:
df_mcs.describe()

In [ ]:
print(f'mean cosine similarity: {mcs_score}')

df_cb_evaluation_results = evaluate_model_and_append_to_df(lmf_model, csr_train, csr_test, 'LogisticMatrixFactorization', df_cb_evaluation_results)

In [ ]:
add_mean_cos_similarity(df_cb_evaluation_results, 'LogisticMatrixFactorization', mcs_score)
df_cb_evaluation_results

### BayesianPersonalizedRanking

Bayesian Personalized Ranking (BPR) est un algorithme de filtrage collaboratif qui se concentre sur la création de recommandations de classement en se basant sur les préférences ordonnées des utilisateurs. Il est adapté aux ensembles de données avec des évaluations implicites et est utilisé pour classer les articles en fonction de leur pertinence pour les utilisateurs.

In [ ]:
bpr_model = BayesianPersonalizedRanking(
    factors=32,
    learning_rate=0.05,
    regularization=0.05,
    iterations=50,
)

bpr_model.fit(csr_train)

Utilisons notre modèle pour suggérer des articles à certains utilisateurs.

In [ ]:
user_ids = [15, 706]
recommend_articles_to_users(bpr_model, user_ids, csr_test)

__Evaluation du modèle BayesianPersonalizedRanking__

Evaluons à quel point les recommandations d'articles sont similaires aux articles déjà consultés par les utilisateurs. Pour cela on utilise des embeddings d'articles et la similarité cosinus pour mesurer cette similarité.

In [ ]:
mcs_score, df_mcs = calculate_user_recommendation_similarity(bpr_model, df_test, csr_train)
df_mcs.head()

In [ ]:
df_mcs.describe()

In [ ]:
print(f'mean cosine similarity: {mcs_score}')
model_name = 'BayesianPersonalizedRanking'

df_cb_evaluation_results = evaluate_model_and_append_to_df(bpr_model, csr_train, csr_test, model_name, df_cb_evaluation_results)

In [ ]:
add_mean_cos_similarity(df_cb_evaluation_results, model_name, mcs_score)
df_cb_evaluation_results

### Comparaison des modèles

📊 **Comparaison des modèles de recommandation**

L'évaluation des modèles repose sur plusieurs métriques : 

1. **Mean Cosine Similarity (MCS)**  
   - Cette métrique mesure la **similitude moyenne** entre les recommandations et les articles consultés par les utilisateurs.
   - Le **BayesianPersonalizedRanking** obtient la valeur la plus élevée (**0.5120**), indiquant une meilleure correspondance entre les recommandations et les préférences des utilisateurs.

2. **Precision**  
   - Évalue la proportion d'articles recommandés qui sont pertinents.  
   - **AlternatingLeastSquares (ALS)** obtient le **meilleur score (0.0403)**, bien que la précision globale reste faible pour tous les modèles.

3. **MAP (Mean Average Precision)**  
   - Mesure la **qualité du classement** des articles recommandés.  
   - L'**ALS** atteint la **meilleure valeur (0.02514)**, indiquant que les articles pertinents sont mieux positionnés dans les recommandations.

4. **NDCG (Normalized Discounted Cumulative Gain)**  
   - Évalue l’**ordre de classement** des articles pertinents dans les recommandations.  
   - L'**ALS** a la **meilleure valeur (0.03099)**, signifiant que les recommandations sont globalement bien ordonnées.

---

### **🏆 Quel est le meilleur modèle ?**
Sur la base de ces résultats :
- **AlternatingLeastSquares (ALS) semble être le modèle le plus performant globalement**.
- Il obtient le **meilleur score en précision, MAP et NDCG**, ce qui signifie qu'il classe mieux les articles les plus pertinents pour l'utilisateur.
- Cependant, **BayesianPersonalizedRanking** a une **meilleure Mean Cosine Similarity**, suggérant qu'il génère des recommandations plus proches des préférences des utilisateurs.

**🎯 Recommandation :**  
Si l’objectif est **d’améliorer la pertinence des recommandations**, **AlternatingLeastSquares** est **le choix optimal**.  
Toutefois, **BayesianPersonalizedRanking** pourrait être exploré pour un modèle hybride combinant les avantages des deux approches.

In [ ]:
als_model.save('models/als_implicit_model.npz')

## 4.2. Recommandation basée sur le contenu <a name="modelisation-content">

Pour la recommandation basée sur le contenu j'ai créé une fonction nommée `recommend_content_based_articles`. Cette fonction recommande des articles à un utilisateur en se basant sur ses lectures récentes et inclut des articles non consultés. Elle prend en compte à la fois la similarité entre les articles et la nouveauté des articles pour fournir des recommandations personnalisées.  
Voici comment elle fonctionne sans entrer dans les détails techniques :

1. La fonction commence par obtenir les articles les plus récemment consultés par l'utilisateur en fonction de la valeur de `user_history_length`.

2. Elle sélectionne un certain nombre d'articles non consultés par l'utilisateur. Le nombre d'articles non consultés dépend de `num_recommendations` modulo 4, ce qui signifie que si num_recommendations est, par exemple, 7, il sélectionnera 3 articles non consultés.

3. Pour chaque article récemment consulté par l'utilisateur, la fonction recherche des articles similaires par rapport à l'article consulté. Ces articles similaires sont ensuite ajoutés au DataFrame des recommandations.

4. Un sous-ensemble aléatoire d'articles similaires est sélectionné à partir des recommandations. La taille de ce sous-ensemble dépend du nombre total de recommandations moins le nombre d'articles non consultés.

5. Enfin, la fonction ajoute les articles non consultés à la liste des recommandations. Ces articles non consultés ont un score de similarité de 0, car ils n'ont pas encore été lus par l'utilisateur.

6. La fonction renvoie la liste finale d'articles recommandés, qui sont une combinaison d'articles similaires aux articles récemment consultés et d'articles non consultés.

In [ ]:
content_based_recommendations = recommend_content_based_articles(df_article_popularity, 0, num_recommendations=5, user_history_length=5)
content_based_recommendations

On affiche ici cinq articles recommandés pour un utilisateur identifié par le numéro 0. Parmi ces cinq articles, quatre sont des recommandations basées sur le contenu des cinq articles les plus récemment consultés par l'utilisateur, tandis qu'un article est nouveau et n'a pas encore été consulté par aucun utilisateur.

## 4.3. Recommandations pour les nouveaux utilisateurs <a name="modelisation-new">

Pour un utilisateur qui n'a pas d'historique de clic, nous allons simplement lui recommander les articles ayant la somme des notes de popularité la plus élevée.

In [ ]:
top_articles = get_top_popular_articles(df_article_popularity, num_recommendations=5)
top_articles

## 4.4. Recommandation collaborative (bibliothèque Surprise) <a name="modelisation-surprise">

Nous explorerons le domaine de la recommandation collaborative en utilisant la bibliothèque "Surprise" (Simple Python RecommendatIon System Engine). La recommandation collaborative se base sur le principe que les utilisateurs partageant des préférences similaires ont tendance à apprécier les mêmes articles. Pour ce faire, nous mettrons en œuvre plusieurs modèles de filtrage collaboratif, chacun ayant ses propres caractéristiques et avantages.

La bibliothèque "Surprise" nous offre une gamme de modèles, dont __SVD__ (Singular Value Decomposition), __BaselineOnly__ et __CoClustering__, qui nous permettront d'explorer différentes approches pour prédire les notes en utilisant les données disponibles.

### SVD

In [ ]:
svd_model = SVD(n_factors=30, lr_all=0.02, reg_all=0.1)
svd_model.fit(trainset)
predictions = svd_model.test(testset)

In [ ]:
df_evaluation_results = add_evaluation_scores('SVD', predictions, df_evaluation_results)
df_evaluation_results

In [ ]:
recommended_articles = recommend_top_articles(svd_model, 15)
recommended_articles

### BaselineOnly

In [ ]:
bonly_model = BaselineOnly()
bonly_model.fit(trainset)
predictions = bonly_model.test(testset)

In [ ]:
df_evaluation_results = add_evaluation_scores('BaselineOnly', predictions, df_evaluation_results)
df_evaluation_results

In [ ]:
recommended_articles = recommend_top_articles(bonly_model, 15)
recommended_articles

### CoClustering

In [ ]:
coclustering_model = CoClustering()
coclustering_model.fit(trainset)
predictions = coclustering_model.test(testset)

In [ ]:
df_evaluation_results = add_evaluation_scores('CoClustering', predictions, df_evaluation_results)
df_evaluation_results

### 📊 **Comparaison des modèles de recommandation**

L'évaluation des modèles repose sur trois métriques :  

1. **RMSE (Root Mean Square Error)**  
   - Évalue l'écart entre les prédictions et les valeurs réelles.  
   - Le **SVD** obtient le **meilleur score (0.0732)**, indiquant des prédictions plus précises.

2. **MSE (Mean Squared Error)**  
   - Indique la variance des erreurs de prédiction.  
   - **SVD a le plus faible MSE (0.0054)**, suivi par **BaselineOnly (0.0114)** et **CoClustering a le plus élevé (0.1318)**.

3. **MAE (Mean Absolute Error)**  
   - Mesure l'erreur moyenne en valeur absolue.  
   - **SVD affiche le score le plus bas (0.0625)**, suivi par **BaselineOnly (0.0979)**, alors que **CoClustering est nettement plus élevé (0.3265)**.

---

### **🏆 Quel est le meilleur modèle ?**
Sur la base de ces résultats :
- **SVD est le modèle le plus performant**, obtenant les **meilleurs scores en RMSE, MSE et MAE**, ce qui signifie qu’il offre des **prédictions plus précises et plus fiables**.
- **BaselineOnly est un modèle de référence**, fournissant des performances correctes mais inférieures à celles de SVD.
- **CoClustering affiche des erreurs nettement plus élevées**, suggérant qu'il n'est pas optimal pour cette tâche spécifique.

**🎯 Recommandation :**  
Le modèle **SVD est le plus adapté** pour cette application de recommandation, car il minimise les erreurs et génère des **prédictions plus précises**.  
Cependant, il pourrait être intéressant d’**explorer une approche hybride**, combinant les forces de différents modèles pour améliorer encore la qualité des recommandations.

In [ ]:
recommended_articles = recommend_top_articles(coclustering_model, 15)
recommended_articles

# 5. Mise à jour des bases de données <a name="updatedata">

L'objectif principal de la mise à jour des bases de données utilisateurs et articles est de garantir que le système de recommandation puisse offrir des recommandations pertinentes et à jour aux utilisateurs.  
La fréquence de mise à jour des bases de données utilisateurs et articles peut varier en fonction des besoins de la plateforme, mais elle doit être régulière pour maintenir la qualité des recommandations. Elle peut être quotidienne, hebdomadaire ou en temps réel, selon la dynamique de la plateforme.

## 5.1. Gestion des articles <a name="update-articles">

__Besoins de mise à jour pour la base de données des articles__ :

1. __Ajout de nouveaux articles__ : Lorsque de nouveaux articles sont publiés ou ajoutés à la plateforme, ils doivent être intégrés à la base de données des articles.

2. __Mise à jour des informations d'articles__ : Les informations sur les articles, telles que les métadonnées, les catégories, les évaluations, et les interactions, doivent être régulièrement mises à jour pour refléter les changements et les évolutions.

3. __Suppression d'articles obsolètes__ : Les articles qui ne sont plus pertinents, obsolètes, ou qui ont été retirés de la plateforme doivent être supprimés de la base de données.

4. __Gestion des changements de disponibilité__ : Si un article devient indisponible ou est retiré de la plateforme, cette information doit être mise à jour dans la base de données pour éviter de recommander des articles qui ne peuvent pas être consultés.

In [ ]:
# Example usage
new_article_data = {
    'category_id': 3,
    'created_at_ts': 1635798000000,
    'publisher_id': 5,
    'words_count': 750
}

df_articles_metadata = add_new_article(df_articles_metadata, new_article_data)

df_articles_metadata.tail()

## 5.2. Gestion des utilisateurs <a name="update-users">

__Besoins de mise à jour pour la base de données des utilisateurs__ :

1. __Ajout de nouveaux utilisateurs__ : Lorsqu'un nouvel utilisateur s'inscrit ou rejoint la plateforme, son profil et ses préférences doivent être ajoutés à la base de données.

2. __Mise à jour des préférences__ : Les préférences des utilisateurs peuvent évoluer avec le temps. Il est donc nécessaire de permettre aux utilisateurs de mettre à jour leurs informations et préférences, et de refléter ces changements dans la base de données.

3. __Suppression de comptes__ : Les comptes d'utilisateurs inactifs ou désactivés doivent être supprimés de la base de données pour maintenir sa propreté et son efficacité.

# 6. Synthèse

La start-up **My Content** développe une application de **recommandation de contenus** axée sur la lecture. Les données d’interaction des utilisateurs avec les articles de **Globo.com** sont utilisées pour créer un **système de recommandation personnalisé**. Ces données sont particulièrement riches, avec **des millions de clics**, impliquant **des centaines de milliers d’utilisateurs** et **des milliers d’articles**.

Afin de capturer les préférences implicites des utilisateurs, **un indicateur de popularité** a été conçu. Les données ont été divisées en ensembles **d’apprentissage** et de **test**, avec la création de **matrices creuses CSR** pour alimenter les modèles de recommandation.

Nous avons exploré **deux principales approches** :
1. **Le filtrage collaboratif** (basé sur l’historique des interactions des utilisateurs).
2. **Le filtrage basé sur le contenu** (basé sur les caractéristiques des articles).

Chaque méthode présente des avantages et des limites. Nous avons utilisé plusieurs **métriques d’évaluation** :
- **MAP** (Mean Average Precision)
- **NDCG** (Normalized Discounted Cumulative Gain)
- **Précision**
- **Similitude moyenne des recommandations**

### 🏆 **Meilleure méthode et meilleur modèle**
L’évaluation des performances des modèles a permis d’identifier **le filtrage collaboratif comme la méthode la plus performante** pour notre système de recommandation.

Dans cette approche, **le modèle Alternating Least Squares (ALS) s’impose comme le meilleur choix**. Il surpasse les autres modèles en obtenant les **meilleures performances globales** sur **toutes les métriques clés** (précision, MAP, NDCG, et similitude moyenne). Ce modèle est donc retenu comme **référence pour la mise en production** du système de recommandation.

🎯 **Conclusion** : L’approche **collaborative** combinée au **modèle ALS** constitue la solution la plus efficace pour offrir **des recommandations pertinentes et personnalisées** aux utilisateurs de My Content.